# Setup
### Default imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

### Select back-end device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

print(f"Using {device} as default device")

### Dataset

# Model
### Our architecture definition

In [ ]:
from torch import nn

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


#dw
class DepthWiseConvolution(nn.Sequential):
    def __init__(self, in_fts, stride = 1):
        super(DepthWiseConvolution,self).__init__(
            nn.Conv2d(in_fts,in_fts,kernel_size=(3,3),stride=stride,padding=(1,1), groups=in_fts, bias=False),
            nn.BatchNorm2d(in_fts),
            nn.ReLU6(inplace=True))


#pw
class PointWiseConvolution(nn.Sequential):
    def __init__(self,in_fts,out_fts):
        super(PointWiseConvolution,self).__init__(
            nn.Conv2d(in_fts,out_fts,kernel_size=(1,1),bias=False),
            nn.BatchNorm2d(out_fts),
            nn.ReLU6(inplace=True))


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1, norm_layer=None):
        padding = (kernel_size - 1) // 2
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            norm_layer(out_planes),
            nn.ReLU6(inplace=True)
        )



class Bottleneck(nn.Module):
    def __init__(self,inp, oup, stride, expand_ratio, norm_layer=nn.BatchNorm2d):
        super(Bottleneck, self).__init__()
        self.stride = stride

        hidden_dim = int(round(inp*expand_ratio))
        layers = []
        self.use_res_connect = self.stride == 1 and inp == oup

        #pw
        if expand_ratio != 1:
            layers.append(PointWiseConvolution(inp,hidden_dim))

        #dw
        layers.extend([
            DepthWiseConvolution(hidden_dim,stride),
            #pw-linear
            nn.Conv2d(hidden_dim,oup,1,1,0,bias=False),
            nn.BatchNorm2d(oup)])

        self.conv = nn.Sequential(*layers)


    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class OurObjectDetectionNet(nn.Module):
    def __init__(self,bottleneckLayerDetail,inp = 3,num_classes = 50,width_mult = 1.0,round_nearest=8):
        super(OurObjectDetectionNet, self).__init__()

        self.out = None

        bloco = Bottleneck
        inverted_residual_setting = bottleneckLayerDetail

        input_channel = 32
        last_channel = 1280

        input_channel = _make_divisible(input_channel*width_mult,round_nearest)
        self.last_channel = _make_divisible(last_channel*width_mult,round_nearest)

        #first layer
        features = [ConvBNReLU(inp, input_channel, stride=2)]

        #build layers
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c*width_mult,round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(bloco(input_channel,output_channel,stride = stride,expand_ratio=t))
                input_channel = output_channel


        #last layer
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))

        #make sequential
        self.features = nn.Sequential(*features)

        #classificador
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes))

    def __forward_impl(self, x):
        x = self.features(x)
        x = nn.functional.adaptive_avg_pool2d(x,1).reshape(x.shape[0],-1)
        x = self.classifier(x)

        return x

    def forward(self, x):
        x = self.__forward_impl(x)
        return x


### Model declaration

In [ ]:
bottleneckLayerDetail = [
    # t, c, n, s
    [1, 16, 1, 1],
    [6, 24, 2, 2],
    [6, 32, 3, 2],
    [6, 64, 4, 2],
    [6, 96, 3, 1],
    [6, 160, 3, 2],
    [6, 320, 1, 1],
]

our_model = OurObjectDetectionNet(bottleneckLayerDetail)
summary(our_model, (1,3,224,224))

### MobileNet V2

In [ ]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

mobilenet_model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V2)

### Convert model for mobile

In [ ]:
!pip install wget

In [ ]:
import wget
import os.path

if not os.path.isfile("convert.py"):
    wget.download(
        "https://raw.githubusercontent.com/johnpolsh/inf721-tpfinal/main/colab/convert.py",
        "convert.py")

In [ ]:
from convert import convert_for_mobile

convert_for_mobile(mobilenet_model, "object_detection")